In [57]:
import torch
import gymnasium as gym
import numpy as np
import mediapy as media
# 필요한 클래스 및 함수 임포트
from cleanrl.cleanrl.ppo_continuous_action import Agent, Args, ppo_make_env
import cv2
from robosuite.utils.camera_utils import CameraMover

visualize = True
frames = []

# Argument 설정
args = Args()
task_id = 'pickplace'
seed = 0
gamma = 0.99
num_episodes = 1
render_camera = ['birdview']#,'agentview'] #('frontview', 'birdview', 'agentview', 'robot0_robotview', 'robot0_eye_in_hand')
camera_names = render_camera


# 환경 생성
env = gym.vector.SyncVectorEnv(
    [ppo_make_env(
        task_id=task_id, 
        reward_shaping=args.reward_shaping,
        idx=0, 
        control_mode="OSC_POSITION",
        capture_video=False, 
        run_name="eval", 
        gamma= args.gamma, 
        active_rewards="r",
        active_image=True, 
        fix_object=args.fix_object,
        wandb_enabled=False,
        verbose=False,
        control_freq=20,
        render_camera=render_camera,
        camera_names=camera_names,

        )
    ]
)


def colorize_depth(frame):
    # Assuming the depth image is in float32 and contains values representing distances.
    # Normalize the depth image to 0-255 for visualization
    min_depth = np.min(frame)
    max_depth = np.max(frame)
    normalized_depth = 255 * (frame - min_depth) / (max_depth - min_depth)
    normalized_depth = normalized_depth.astype(np.uint8)

    # Apply a colormap for better visualization (COLORMAP_JET is commonly used)
    colorized_depth = cv2.applyColorMap(normalized_depth, cv2.COLORMAP_JET)
    return colorized_depth


# 디바이스 설정 (cuda가 가능하면 cuda 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    print("Using CUDA")
else :
    assert device == torch.device("cpu")

# Agent 초기화 및 모델 불러오기
agent = Agent(env).to(device)
agent.eval()  # 평가 모드로 전환

# 평가 수행
total_rewards = []
viewer_image_key = 'birdview'+'_depth'

# generate samples
num_samples = 10

for i in range(num_samples):
    obs, _ = env.reset()
    obs = torch.Tensor(obs).to(device)
    done = False
    episode_reward = 0

    image_frame = env.envs[0].image_states[viewer_image_key]
    if not viewer_image_key.endswith('depth'):
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환
    else:
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.float32)
    image_frame = colorize_depth(image_frame)
    
    can_pos = env.envs[0].sim.data.get_body_xpos('Can_main')  # Assuming the object is called 'Can'
    can_quat = env.envs[0].sim.data.get_body_xquat('Can_main')
    pos_text = f"Pos: {can_pos}"
    quat_text = f"Quat: {can_quat}"
    cv2.putText(image_frame, pos_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255, 255, 255), 1)
    cv2.putText(image_frame, quat_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255, 255, 255), 1)
   
    
    frames.append(image_frame)

media.show_video(frames, fps=20)


### controller_config: OSC_POSITION ###
#### J PickPlace ####
fix_object:False
start with grasp lock: True
control_freq: 20
ignore_done: False


In [1]:
import torch
import gymnasium as gym
import numpy as np
import mediapy as media
# 필요한 클래스 및 함수 임포트
from cleanrl.cleanrl.ppo_continuous_action import Agent, Args, ppo_make_env
import cv2
from robosuite.utils.camera_utils import CameraMover

visualize = True
frames = []

# Argument 설정
args = Args()
task_id = 'pickplace'
seed = 0
gamma = 0.99
num_episodes = 1
render_camera = ['birdview']#,'agentview'] #('frontview', 'birdview', 'agentview', 'robot0_robotview', 'robot0_eye_in_hand')
camera_names = render_camera


# 환경 생성
env = gym.vector.SyncVectorEnv(
    [ppo_make_env(
        task_id=task_id, 
        reward_shaping=args.reward_shaping,
        idx=0, 
        control_mode="OSC_POSITION",
        capture_video=False, 
        run_name="eval", 
        gamma= args.gamma, 
        active_rewards="r",
        active_image=True, 
        fix_object=args.fix_object,
        wandb_enabled=False,
        verbose=False,
        control_freq=20,
        render_camera=render_camera,
        camera_names=camera_names,

        )
    ]
)

# 디바이스 설정 (cuda가 가능하면 cuda 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    print("Using CUDA")
else :
    assert device == torch.device("cpu")

# 평가 수행
viewer_image_key = 'birdview'+'_depth'

# generate samples
num_samples = 10


depth_frames = []
can_pos_quat = []

for i in range(num_samples):
    obs, _ = env.reset()
    obs = torch.Tensor(obs).to(device)
    done = False
    episode_reward = 0

    image_frame = env.envs[0].image_states[viewer_image_key]
    if not viewer_image_key.endswith('depth'):
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환
    else:
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.float32)
    
    can_pos = env.envs[0].sim.data.get_body_xpos('Can_main')  # Assuming the object is called 'Can'
    can_quat = env.envs[0].sim.data.get_body_xquat('Can_main')
    depth_frames.append(image_frame)
    can_pos_quat.append(np.concatenate([can_pos, can_quat]))
    

[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /research/rs4tmr/robosuite/scripts/setup_macros.py (macros.py:55)


### controller_config: OSC_POSITION ###
#### J PickPlace ####
fix_object:False
start with grasp lock: True
control_freq: 20
ignore_done: False


In [3]:
# save the depth frames and can pos/quat pairs

import os
import pickle
from my_utils import get_current_time

data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
depth_frames = np.array(depth_frames)
can_pos_quat = np.array(can_pos_quat)

# time stamp
time_stamp = 1

depth_frames_path = os.path.join(data_dir, f'depth_frames_{time_stamp}.npy')
can_pos_quat_path = os.path.join(data_dir, f'can_pos_quat_{time_stamp}.npy')



np.save(depth_frames_path, depth_frames)
np.save(can_pos_quat_path, can_pos_quat)

print(f"Depth frames saved to {depth_frames_path}")

Depth frames saved to data/depth_frames_1.npy
